# CS 329 E Data Visualization Homework 5

## National Parks and Biodiversity 

### Submitted by : Maxwell Kretschmer 11am

In our last homework, we'll combine some of the concepts from the 
recent lab (geo data) with merging data and create an interactive 
dashboard. 


In [1]:
!pip install geopandas

In [2]:
# importing packages

# data wrangling
import pandas as pd
import numpy as np

# plotting
import altair as alt

# working with geographic data
import json
import geopandas as gpd
from vega_datasets import data

## Loading in a shape file
This zip file contains a _shape_ file.  This is one of the types of geometry data supported by geopandas ([doc](https://geopandas.org/en/stable/docs/user_guide/io.html?highlight=shape%20files#reading-spatial-data))

In [3]:
# importing shapefile - We can use a zip file that contains a shape file by reading it from a URL
gdf = gpd.read_file('http://cs.utexas.edu/~chaney/nps_boundary.zip')

In [4]:
gdf.sample(5)

,UNIT_CODE,GIS_Notes,UNIT_NAME,DATE_EDIT,STATE,REGION,GNIS_ID,UNIT_TYPE,CREATED_BY,METADATA,PARKNAME,Shape_Leng,Shape_Area,Unified_Re,Old_Region,geometry
106,CORO,Lands - http://landsnet.nps.gov/tractsnet/docu...,Coronado National Memorial,2011-02-14,AZ,IM,40512,National Memorial,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Coronado,0.193931,0.001846,8,IM,"POLYGON ((-110.22406 31.34005, -110.22406 31.3..."
371,WHSA,Lands - http://landsnet.nps.gov/tractsnet/docu...,White Sands National Park,2020-01-22,NM,IM,914261,National Park,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,White Sands,1.179904,0.057737,7,IM,"POLYGON ((-106.17077 32.81114, -106.17082 32.8..."
281,CUIS,Lands - http://landsnet.nps.gov/tractsnet/docu...,Cumberland Island National Seashore,2018-10-16,GA,SE,325829,National Seashore,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Cumberland Island,0.678421,0.014641,2,SE,"MULTIPOLYGON (((-81.54881 30.72022, -81.54885 ..."
175,JODA,Lands - http://landsnet.nps.gov/tractsnet/docu...,John Day Fossil Beds National Monument,2015-07-08,OR,PW,1133439,National Monument,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,John Day Fossil Beds,0.858152,0.006420,9,PW,"MULTIPOLYGON (((-119.61816 44.60008, -119.6181..."
333,PERL,Lands - http://landsnet.nps.gov/tractsnet/docu...,Pearl Harbor National Memorial,2019-08-29,HI,PW,need,National Memorial,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Pearl Harbor,0.030530,0.000008,12,PW,"MULTIPOLYGON (((-157.95220 21.36360, -157.9521..."


## Q1 - Overlay the National Parks and Monuments on a county map of the USA
Using the geopandas data frame `gdf`, filter out just the national parks and monuments and overlay
them on a county map of the USA.  Use the Vega data set to map the counties, using [this page](https://altair-viz.github.io/gallery/choropleth.html) as a reference.  Create a tool tip that shows the park name (`UNIT_NAME`) and region (`REGION`) of the park, title your chart, and encode the color with the Region of the park using the `dark2` colorscheme. See [this page](https://altair-viz.github.io/user_guide/customization.html#customizing-colors) for a reference on changing the color scheme. 

This would be an appropriate visualization if our user wanted to understand the landsize of the parks and how the regions were labeled. 

In [5]:
 # your code here
counties = alt.topo_feature(data.us_10m.url, 'counties')
natPM = gdf[gdf['UNIT_TYPE'].isin(['National Park', 'National Monument'])]
natPMR = natPM[natPM['REGION'].isin(['IM','PW'])]

countiesM = alt.Chart(counties).mark_geoshape().encode(
    color=alt.value('grey')
).project(
    type='albersUsa'
).properties(
    width=750, #1000
    height=500 #700
)
nationalPM = alt.Chart(natPMR).mark_geoshape().encode(
    color=alt.Color('REGION', scale=alt.Scale(scheme='dark2')),
    tooltip=['UNIT_NAME','REGION']
).project(
    type='albersUsa'
).properties(
    width=750,
    height=500
)
countiesM + nationalPM

Output hidden; open in https://colab.research.google.com to view.

## Loading in a csv a species data

In [6]:
df_species = pd.read_csv('http://cs.utexas.edu/~chaney/numspecies.csv')
df_species = df_species.fillna( value = 0 ) # Fill the missing species counts with zeros for visualization
df_species.sample(5)

,UNIT_CODE,All Types,Amphibian,Bird,Fish,Mammal,Reptile,Vascular Plant,Fungi,Insect,Invertebrate,Spider/Scorpion,Nonvascular Plant,Algae,Crab/Lobster/Shrimp,Slug/Snail
21,GRBA,2653,22.0,330,21.0,103,58.0,1549,15.0,456.0,29.0,18.0,16.0,0.0,6.0,30.0
31,JOTR,2294,5.0,301,1.0,67,52.0,1314,38.0,342.0,10.0,153.0,6.0,5.0,0.0,0.0
8,CARE,1566,6.0,248,15.0,73,21.0,1203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,CRLA,3760,20.0,252,13.0,96,20.0,1628,192.0,994.0,68.0,17.0,193.0,218.0,40.0,9.0
44,REDW,6310,33.0,501,247.0,154,39.0,2257,1363.0,744.0,334.0,7.0,251.0,111.0,122.0,147.0


## Q2 - Change data to Long format 
Remember [long form vs wide form formats](https://altair-viz.github.io/user_guide/data.html#long-form-vs-wide-form-data)? It turns out our species data is wide form.  Convert it to long form and save the new dataframe to `df_species_long`. Name your new columns `Species Type` and `Species Count` so they look pretty when we plot them later.

In [7]:
 # your code here
df_species_long = df_species.melt('UNIT_CODE', var_name='Species Type', value_name='Species Count')

In [8]:
# sniff check
df_species_long.sample(5)

,UNIT_CODE,Species Type,Species Count
449,ARCH,Insect,0.0
433,OLYM,Fungi,0.0
174,BRCA,Fish,1.0
312,KATM,Reptile,0.0
446,YOSE,Fungi,0.0


## Q3 - Get lat/long and area from park shape
We have the shape file for the national parks, which contains the polygon for the park boundary, but we decide for the visualization we want to encode the location with a circle whose size is constant.  We also are interested in comparing the area of the park with the species diversity to discover trends. To accomplish these tasks we need to calculate the centroid of the park to get a single lat/long and calculate the area of the polygon in acres (a convenient unit for the USA). 

To do math in geopandas you have to translate between "EPSG:4269" and "EPSG:5070" (USA Albers projection)

Add the `Acres` column to the `gdf` geopandas dataframe, and replace the `geometry` column with the centroid. Use the [intro to geopandas](https://geopandas.org/en/stable/getting_started/introduction.html) documentation to help with the syntax.

Check that everything looks OK by plotting the lat/long of the parks on top of the county map as before. Include a tooltip that shows the region, name of park, and Acres. Use the `dark2` color scheme.

I'm helping you out with the coordinate reference systems here; note that there are many many map reference coordinates and we can see the reference for a given geopandas dataframe with the `crs` property. 

In [9]:
gdf = gdf.to_crs( 'EPSG:5070') 

In [10]:
# look at the coordinate reference of the data - this needs to be translated to do math, but then put back
gdf.crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [11]:
# Find the area in acres
gdf['Acres'] = gdf.area / 4046.8564224 # your code here to calculate acrage 

In [12]:
# sniff check
# The value from Rocky Mountain National Park webpage is 265,807 acres and our estimate is off probably due to the 
# projection we used was not locally optimal, or maybe the shape file wasn't exact. If we were a GIS class
# this would probably bother us, but for now we'll call this close enough (your number should be about 0.5% off)
gdf[gdf['UNIT_CODE'] == 'ROMO']['Acres']
# (267041.649602 - 265807) / 265807 = 0.00464491003 * 100 = 0.464491004%

415    267041.649602
Name: Acres, dtype: float64

In [13]:
gdf['geometry'] = gdf.centroid # your code here to convert the polygons data to point data 

In [14]:
gdf = gdf.to_crs( 'EPSG:4269') # This takes us back to lat/long space
gdf.sample(5)

,UNIT_CODE,GIS_Notes,UNIT_NAME,DATE_EDIT,STATE,REGION,GNIS_ID,UNIT_TYPE,CREATED_BY,METADATA,PARKNAME,Shape_Leng,Shape_Area,Unified_Re,Old_Region,geometry,Acres
326,AMIS,Lands - http://landsnet.nps.gov/tractsnet/docu...,Amistad National Recreation Area,2019-07-22,TX,IM,1329499,National Recreation Area,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Amistad,10.532744,0.023235,6,IM,POINT (-101.09738 29.54806),61681.641639
276,TUMA,Lands - http://landsnet.nps.gov/tractsnet/docu...,Tumacacori National Historical Park,2018-04-05,AZ,IM,35519,National Historical Park,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,TumacÃ¡cori,0.081460,0.000139,8,IM,POINT (-111.03847 31.56116),362.502556
93,MORU,Lands - http://landsnet.nps.gov/tractsnet/docu...,Mount Rushmore National Memorial,2010-12-13,SD,MW,1262098,National Memorial,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Mount Rushmore,0.133398,0.000577,5,MW,POINT (-103.45252 43.88037),1273.622837
383,PETR,Lands - http://landsnet.nps.gov/tractsnet/docu...,Petroglyph National Monument,2020-06-18,NM,IM,920404,National Monument,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Petroglyph,0.581944,0.002883,7,IM,POINT (-106.74916 35.13915),7202.153284
266,SARA,Lands - http://landsnet.nps.gov/tractsnet/docu...,Saratoga National Historical Park,2021-03-23,NY,NE,964487,National Historical Park,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Saratoga,0.299266,0.001588,1,NE,POINT (-73.62992 42.99831),3553.851832


In [15]:
# your map code here
natPM = gdf[gdf['UNIT_TYPE'].isin(['National Monument', 'National Park'])]

counties = alt.topo_feature(data.us_10m.url, 'counties')
countiesM = alt.Chart(counties).mark_geoshape().encode(
    color=alt.value('#999999')
).project(
    type='albersUsa'
).properties(
    width=1000,
    height=600
)
nationalPM = alt.Chart(natPM).mark_geoshape().encode(
    color=alt.Color('REGION', scale=alt.Scale(scheme='dark2')),
    tooltip=['UNIT_NAME','REGION']
).project(
    type='albersUsa'
).properties(
    width=1000,
    height=600
)
countiesMap = countiesM + nationalPM
countiesMap

alt.LayerChart(...)

## Q4 - Merge Geo Data and and Species Data
We've been using the `UNIT_NAME` for the park name in the plots, and we want to use that same name, and the acerage
when plotting the species data.  Add these two columns to our `df_species_long` dataframe using a inner join, renaming the `UNIT_NAME` to `Park Name`, and `REGION` to `Region` for pretty 
visualization. The key for the merge is `UNIT_CODE`. Save this new dataframe as `df_species_final`

In [16]:
natPM.head(5)

,UNIT_CODE,GIS_Notes,UNIT_NAME,DATE_EDIT,STATE,REGION,GNIS_ID,UNIT_TYPE,CREATED_BY,METADATA,PARKNAME,Shape_Leng,Shape_Area,Unified_Re,Old_Region,geometry,Acres
2,LIBI,Lands - http://landsnet.nps.gov/tractsnet/docu...,Little Bighorn Battlefield National Monument,2005-02-23,MT,IM,806297,National Monument,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Little Bighorn Battlefield,0.148934,0.000364,5,IM,POINT (-107.41834 45.55632),780.020777
3,CAVO,Lands - http://landsnet.nps.gov/tractsnet/docu...,Capulin Volcano National Monument,2004-07-22,NM,IM,936867,National Monument,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Capulin Volcano,0.087358,0.000323,7,IM,POINT (-103.97104 36.78171),790.596679
5,GICL,Lands - http://landsnet.nps.gov/tractsnet/docu...,Gila Cliff Dwellings National Monument,2004-05-10,NM,IM,928945,National Monument,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Gila Cliff Dwellings,0.076633,0.000237,7,IM,POINT (-108.27067 33.22715),605.342635
6,TUZI,Lands - http://landsnet.nps.gov/tractsnet/docu...,Tuzigoot National Monument,2003-11-21,AZ,IM,35601,National Monument,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Tuzigoot,0.089346,0.000324,8,IM,POINT (-112.02791 34.77296),812.650731
8,CASA,Lands - http://landsnet.nps.gov/tractsnet/docu...,Castillo de San Marcos National Monument,2005-02-28,FL,SE,307911,National Monument,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Castillo de San Marcos,0.013992,0.000008,2,SE,POINT (-81.31217 29.89786),20.044317


In [17]:
mergeGDF = gdf[['UNIT_CODE','UNIT_NAME', 'Acres','REGION']]
df_species_final = pd.merge(df_species_long, mergeGDF, on="UNIT_CODE")
  
df_species_final.rename(columns = {'UNIT_NAME':'Park Name', 
'REGION':'Region'}, inplace = True)

df_species_final

,UNIT_CODE,Species Type,Species Count,Park Name,Acres,Region
0,ACAD,All Types,1709.0,Acadia National Park,34766.266880,NE
1,ACAD,Amphibian,15.0,Acadia National Park,34766.266880,NE
2,ACAD,Bird,364.0,Acadia National Park,34766.266880,NE
3,ACAD,Fish,38.0,Acadia National Park,34766.266880,NE
4,ACAD,Mammal,55.0,Acadia National Park,34766.266880,NE
...,...,...,...,...,...,...
925,ZION,Spider/Scorpion,0.0,Zion National Park,148709.057703,IM
926,ZION,Nonvascular Plant,0.0,Zion National Park,148709.057703,IM
927,ZION,Algae,0.0,Zion National Park,148709.057703,IM
928,ZION,Crab/Lobster/Shrimp,0.0,Zion National Park,148709.057703,IM


## Q5 - Compare the Acres to Species Count

On a log/log scale compare the total species count (only the rows where "Species Type" is "All Types") with the park size. Encode the region using the same color scale as above. Add a tool tip to see the park name, Species Count, and Acres.

Add a `selection_multi` that will highlight the selected data by turning unselected data light gray.

In [18]:
# your code here
selector = alt.selection_multi(bind='legend',fields=['UNIT_CODE','Park Name'])
speciesCt = df_species_final[df_species_final['Species Type'] == 'All Types']
alt.Chart(speciesCt).mark_circle(size=75).encode(
    x=alt.X("Acres",scale=alt.Scale(type="log")),
    y=alt.Y("Species Count",scale=alt.Scale(type="log")),
    color=alt.condition(selector, 'Region:N', alt.value('lightgray'), scale=alt.Scale(scheme='dark2')),
    tooltip=['Park Name', 'Species Count', 'Acres']
).properties(
    width=400,
    height=300,
    title="Park Size vs Species Count (log/log scale)"
).add_selection(
    selector
)

alt.Chart(...)

## Q6 - Create a rug plot of Species Diversity
Look at how the different Species categores (all Species types that are NOT 'All Types' are distributed
across all the parks.  

Use the same color encoding for the Region as the prior plots. 

Add a `selection_multi` that will highlight the selected data for a park by turning unselected data light gray. 
Also, make the tick a little bigger when it is selected to help find the park in the other categories.

In [19]:
# your code here
speciesTypes = df_species_final[df_species_final['Species Type'] != 'All Types']

selector = alt.selection_multi(bind='legend',fields=['UNIT_CODE','Park Name'])

alt.Chart(speciesTypes).mark_tick(thickness=2).encode(
    x=alt.X("Species Count",scale=alt.Scale(type="sqrt")),
    y=alt.Y("Species Type", title=""),
    color=alt.condition(selector, 'Region:N', alt.value('lightgray'), scale=alt.Scale(scheme='dark2')),
    size=alt.condition(selector, alt.value(20), alt.value(15)),
    tooltip=['Park Name', 'Species Count', 'Species Type', 'Region']
).properties(
    width=400,
    height=300,
    title="Species Count by Type"
).add_selection(
    selector
)

alt.Chart(...)

## Q7 - Create a sorted bar chart of Species Count with a Region Drop down selector

Create a bar chart with horizontal bars that shows the total species count per park with the color channel encoding the Region.

Add a drop down selector so that you can filter the bar chart by just one region. See [this documentation](https://altair-viz.github.io/gallery/multiple_interactions.html) for an example of adding a drop down menu and using it to filter your selections.

In addition to the drop down, include a `selection_multi` that will highlight the selected data by turning unselected data light gray.

In [20]:
# your code here
regions = list(speciesCt['Region'].unique())

speciesCt = df_species_final[df_species_final['Species Type'] == 'All Types']

selector = alt.selection_multi(name="Park Selector", bind='legend',fields=['UNIT_CODE','Park Name'])

dropdown = alt.binding_select(options=[None]+regions, labels=["All"]+regions, name="Filter by a Region:  ")
dropdown_select = alt.selection_single(fields=['Region'], bind=dropdown)

alt.Chart(speciesCt).mark_bar().encode(
    x=alt.X("Species Count",scale=alt.Scale(type="sqrt")),
    y=alt.Y("Park Name", sort='-x', title=""),
    color=alt.condition(selector, 'Region:N', alt.value('lightgray'), scale=alt.Scale(domain=["NE", "IM", "MW", "SE", "PW", "AK"], scheme='dark2') ),
    tooltip=['Park Name', 'Species Count', 'Species Type']
).properties(
    width=300,
    title="Parks Sorted by Species Count"
).add_selection(
    dropdown_select
).transform_filter(
    dropdown_select
).add_selection(
    selector
)

alt.Chart(...)

## Q8 - Bring all the charts together for a Dashboard

Bring all 4 charts and the region filter together.  Any click on one chart should highlight that data in the
other charts.  The colors for the regions should not change depending on the region selection and be consistent across 
all graphs. When plotting your geopandas data frame, filter out only the rows that have species data before making the map. I used the [documentation on customizing titles](https://altair-viz.github.io/user_guide/configuration.html#config-title) to change the font size and name my dashboard (all of the titles in the component graphs I changed to be `subtitles`)

Change your color scheme to `tableau10`.

Hint: There will be only one `selection_multi` that links all the selection highlighting in the charts together.

In [21]:
# VARS
speciesCt = df_species_final[df_species_final['Species Type'] == 'All Types']
speciesTypes = df_species_final[df_species_final['Species Type'] != 'All Types']

regions = list(speciesCt['Region'].unique())

dropdown = alt.binding_select(options=[None]+regions, labels=["All"]+regions, name="Filter by a Region:  ")
dropdown_select = alt.selection_single(fields=['Region'], bind=dropdown)

selector = alt.selection_multi(name="Park Selector", bind='legend',fields=['UNIT_CODE','Park Name'])

gdf.rename(columns = {'UNIT_NAME':'Park Name', 'REGION':'Region'}, inplace = True)

natParks = gdf[gdf['Park Name'].isin(list(speciesCt['Park Name'].unique()))]

counties = alt.topo_feature(data.us_10m.url, 'counties')

regionScale = alt.Scale(domain=["NE", "IM", "MW", "SE", "PW", "AK"], scheme='tableau10') 

In [22]:
# GEOSHAPE
countiesM = alt.Chart(counties).mark_geoshape().encode(
    color=alt.value('#999999')
).project(
    type='albersUsa'
).properties(
    width=850,
    height=600,
    title={
      "text": [""], 
      "subtitle": ["National Parks in the USA"],
    }
)
nationalPM = alt.Chart(natParks).mark_geoshape().encode(
    color=alt.condition(selector, 'Region:N', alt.value('lightgray'), scale=regionScale),
    tooltip=['Park Name','Region','Acres'] # acres format='d'
).project(
    type='albersUsa'
).add_selection(
    selector
).transform_filter(
    dropdown_select
)
countiesMap = countiesM + nationalPM

In [23]:
# ACRES
acreMap = alt.Chart(speciesCt).mark_circle(size=75).encode(
    x=alt.X("Acres",scale=alt.Scale(type="log")),
    y=alt.Y("Species Count",scale=alt.Scale(type="log")),
    color=alt.condition(selector, 'Region:N', alt.value('lightgray'), scale=regionScale),
    tooltip=['Park Name', 'Species Count', 'Acres'] # acres format='d'
).properties(
    width=380,
    title={
      "text": [""], 
      "subtitle": ["Park Size vs Species Count (log/log scale)"],
    }
).add_selection(
    selector
)

In [24]:
# RUG PLOT
speciesTypeMap = alt.Chart(speciesTypes).mark_tick(thickness=2).encode(
    x=alt.X("Species Count",scale=alt.Scale(type="sqrt")),
    y=alt.Y("Species Type", title=""),
    color=alt.condition(selector, 'Region:N', alt.value('lightgray'), scale=regionScale),
    size=alt.condition(selector, alt.value(20), alt.value(15)),
    tooltip=['Park Name', 'Species Count', 'Species Type', 'Region']
).properties(
    width=380,
    title={
      "text": [""], 
      "subtitle": ["Species Count by Type"],
    }
).add_selection(
    selector
)

In [25]:
# BAR CHART
speciesCountMap = alt.Chart(speciesCt).mark_bar().encode(
    x=alt.X("Species Count",scale=alt.Scale(type="sqrt")),
    y=alt.Y("Park Name", sort='-x', title=""),
    color=alt.condition(selector, 'Region:N', alt.value('lightgray'), scale=regionScale ),
    tooltip=['Park Name', 'Species Count', 'Species Type']
).properties(
    width=300,
    title={
      "text": [""], 
      "subtitle": ["Parks Sorted by Species Count"],
    }
).add_selection(
    dropdown_select
).transform_filter(
    dropdown_select
).add_selection(
    selector
)

In [26]:
result = speciesCountMap | (countiesMap & (speciesTypeMap | acreMap))

resultGraph = result.properties(
    title="National Park Diversity Dashboard"
).configure_title(
    fontSize=20,
    subtitleFontSize=16
)
resultGraph

alt.HConcatChart(...)